In [ ]:
#đọc dữ liệu
import pandas as pd

citizen_df = pd.read_csv("../data_clean/citizen_info_clean.csv")
feedback_df = pd.read_csv("../data_clean/feedback_log_clean.csv")
resolution_df = pd.read_csv("../data_clean/resolution_time_clean.csv")

feedback_df['submit_time'] = pd.to_datetime(feedback_df['submit_time'])
resolution_df['resolved_time'] = pd.to_datetime(resolution_df['resolved_time'])

In [ ]:
#3.1 Số lượng phản ánh trung bình theo area

    #Gộp feedback với area
feedback_area = feedback_df.merge(
    citizen_df[['citizen_id', 'area']],
    on='citizen_id',
    how='left'
)
#đếm phản ánh theo area x category
area_category_count = (
    feedback_area
    .groupby(['area', 'category'])
    .size()
    .reset_index(name='feedback_count')
)
 # tính trung bình theo area
avg_feedback_area_category = (
    area_category_count
    .groupby('area')['feedback_count']
    .mean()
    .reset_index(name='avg_feedback_per_category')
    .sort_values('avg_feedback_per_category', ascending=False)
)

avg_feedback_area_category

In [ ]:
#3.2 thời gian xử lý trung bình theo department
#Gộp feedback + resolution
feedback_resolution = feedback_df.merge(
    resolution_df[['feedback_id', 'resolution_hours']],
    on='feedback_id',
    how='left'
)
#Groupby department
avg_resolution_by_department = (
    feedback_resolution
    .groupby('department')['resolution_hours']
    .mean()
    .reset_index(name='avg_resolution_hours')
    .sort_values('avg_resolution_hours', ascending=False)
)

avg_resolution_by_department



In [ ]:
#3.4So sánh thời gian xử lý trung bình giữa các channel
avg_resolution_by_channel = (
    feedback_resolution
    .groupby('channel')['resolution_hours']
    .mean()
    .reset_index(name='avg_resolution_hours')
    .sort_values('avg_resolution_hours', ascending=False)
)

avg_resolution_by_channel


In [ ]:
# 3.5 Xác định area hoặc department có thời gian xử lý cao bất thường
#Theo department
mean_hr = resolution_df['resolution_hours'].mean()
std_hr = resolution_df['resolution_hours'].std()

abnormal_threshold = mean_hr + 1.5 * std_hr

abnormal_departments = (
    feedback_resolution
    .groupby('department')['resolution_hours']
    .mean()
    .reset_index(name='avg_resolution_hours')
)

abnormal_departments = abnormal_departments[
    abnormal_departments['avg_resolution_hours'] > abnormal_threshold
]

abnormal_departments


In [ ]:
#Theo area
feedback_area_resolution = feedback_area.merge(
    resolution_df[['feedback_id', 'resolution_hours']],
    on='feedback_id',
    how='left'
)

abnormal_areas = (
    feedback_area_resolution
    .groupby('area')['resolution_hours']
    .mean()
    .reset_index(name='avg_resolution_hours')
)

abnormal_areas = abnormal_areas[
    abnormal_areas['avg_resolution_hours'] > abnormal_threshold
]

abnormal_areas
